**1. Algoritmo de la suma**

In [ ]:
def sumar(numero1, numero2, base=10):
    # Convertir las cadenas a listas de dígitos (enteros)
    digitos1 = [int(d) for d in reversed(numero1)]
    digitos2 = [int(d) for d in reversed(numero2)]

    # Asegurar que ambas listas tienen la misma longitud
    longitud_maxima = max(len(digitos1), len(digitos2))
    digitos1 += [0] * (longitud_maxima - len(digitos1))
    digitos2 += [0] * (longitud_maxima - len(digitos2))

    # Verificar límite de dígitos
    if longitud_maxima > 20:
        return "Error: Número demasiado grande (máximo 20 dígitos)"

    resultado = []
    acarreo = 0

    # Sumar dígito por dígito
    for d1, d2 in zip(digitos1, digitos2):
        suma = d1 + d2 + acarreo
        resultado.append(suma % base)
        acarreo = suma // base

    # Añadir el acarreo final si existe
    if acarreo > 0:
        resultado.append(acarreo)

    # Convertir el resultado a cadena (invertir el orden)
    resultado_str = ''.join(map(str, reversed(resultado)))

    return resultado_str

# Ejemplo:
numero_a = "12345678901234567890"
numero_b = "9876543210987654321"
print(f"Suma: {numero_a} + {numero_b} = {sumar(numero_a, numero_b)}")

Suma: 12345678901234567890 + 9876543210987654321 = 22222222112222222211


**2. Algoritmo de la resta**

In [ ]:
%%writefile resta_big.c
#include <stdio.h>
#include <string.h>
#include <stdlib.h>

#define MAX_DIGITOS 20

//comparamos dos cadenas numéricas como enteros
int comparar(const char *a, const char *b) {
    int len_a = strlen(a);
    int len_b = strlen(b);
    if (len_a != len_b) return len_a - len_b;
    return strcmp(a, b);  // strcmp devuelve < 0 si a < b, 0 si a == b, > 0 si a > b
}

void resta(const char *u_str, const char *v_str, int base) {	//base 10
    int u[MAX_DIGITOS] = {0}, v[MAX_DIGITOS] = {0}, w[MAX_DIGITOS] = {0};
    int negativo = 0;

    //si u < v, intercambiamos u y v y ponemos el signo negativo
    if (comparar(u_str, v_str) < 0) {
        const char *temp = u_str;
        u_str = v_str;
        v_str = temp;
        negativo = 1;
    }
    int len_u = strlen(u_str), len_v = strlen(v_str);

    for (int i = 0; i < len_u; i++) u[i] = u_str[len_u - 1 - i] - '0';	// restamos '0' (ASCII) para convertir de manera segura el carácter dígito al l valor numérico
    for (int i = 0; i < len_v; i++) v[i] = v_str[len_v - 1 - i] - '0';

    int k = 0;
    for (int i = 0; i < MAX_DIGITOS; i++) {
        int dif = u[i] - v[i] + k;
        if (dif < 0) {
            w[i] = (dif + base) % base;	//hacemos módulo para asegurarnos que está entre 0 y base-1
            k = -1;
        } else {
            w[i] = dif % base;
            k = 0;
        }
    }

    int inicio = MAX_DIGITOS - 1;
    while (inicio > 0 && w[inicio] == 0) inicio--;	//borramos los ceros a la izquierda

    printf("Resultado: ");
    if (negativo) printf("-");
    for (int i = inicio; i >= 0; i--) printf("%d", w[i]);
    printf("\n");
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        fprintf(stderr, "Uso: %s <u> <v>\n", argv[0]);
        return 1;
    }
    resta(argv[1], argv[2], 10);
    return 0;
}


Overwriting resta_big.c


In [ ]:
!gcc resta_big.c -o resta_big


In [ ]:
!./resta_big 13 24


Resultado: -11


**3. Multiplicación**

In [2]:
def multiplicar(numero1, numero2, base=10):
    # Verificar límite de dígitos
    if len(numero1) > 20 or len(numero2) > 20:
        return "Error: Número demasiado grande (máximo 20 dígitos)"

    # Convertir las cadenas a listas de dígitos (enteros), en orden invertido
    digitos1 = [int(d) for d in reversed(numero1)]
    digitos2 = [int(d) for d in reversed(numero2)]

    # Crear una lista para el resultado (máximo len1 + len2 dígitos)
    resultado = [0] * (len(digitos1) + len(digitos2))

    # Multiplicar dígito por dígito
    for i in range(len(digitos1)):
        for j in range(len(digitos2)):
            producto = digitos1[i] * digitos2[j]
            resultado[i + j] += producto
            # Manejo del acarreo
            if resultado[i + j] >= base:
                resultado[i + j + 1] += resultado[i + j] // base
                resultado[i + j] %= base

    # Eliminar ceros finales innecesarios (del lado izquierdo)
    while len(resultado) > 1 and resultado[-1] == 0:
        resultado.pop()

    # Convertir el resultado a cadena (orden correcto)
    resultado_str = ''.join(map(str, reversed(resultado)))

    return resultado_str

# Ejemplo:
numero_a = "45345678401294563890"
numero_b = "93765632109896545210"
print(f"Multiplicación: {numero_a} × {numero_b} = {multiplicar(numero_a, numero_b)}")


Multiplicación: 45345678401294563890 × 93765632109896545210 = 4251866198749467797816217043064618466900


**4. División**

In [ ]:
BASE = 10

def cadena_a_digitos(cadena):
    return [int(c) for c in reversed(cadena.strip())]

def digitos_a_cadena(lista):
    # Elimina ceros altos
    while len(lista) > 1 and lista[-1] == 0:
        lista.pop()
    return ''.join(str(d) for d in reversed(lista))

def multiplicar_por_digito(dividendo, d):
    acarreo = 0
    for i in range(len(dividendo)):
        prod = dividendo[i] * d + acarreo
        dividendo[i] = prod % BASE
        acarreo = prod // BASE
    if acarreo > 0:
        dividendo.append(acarreo)

def restar_producto(dividendo, indice, divisor, coc_prueba):

    prestamo = 0
    acarreo = 0
    n = len(divisor)
    # Resta v * qt
    for k in range(n):
        acarreo += divisor[k] * coc_prueba
        dig = acarreo % BASE
        acarreo //= BASE
        pos = indice + k
        diff = dividendo[pos] - dig - prestamo
        if diff < 0:
            diff += BASE
            prestamo = 1
        else:
            prestamo = 0
        dividendo[pos] = diff

    # Restar acarreo remanente
    pos = indice + n
    while acarreo or prestamo:
        cur = dividendo[pos] - (acarreo % BASE) - prestamo
        acarreo //= BASE
        if cur < 0:
            dividendo[pos] = cur + BASE
            prestamo = 1
        else:
            dividendo[pos] = cur
            prestamo = 0
        pos += 1

    return prestamo == 1

def dividir_cociente_prueba(dividendo0, divisor0):
    # Copiar listas de dígitos
    dividendo = dividendo0[:]
    divisor   = divisor0[:]
    n = len(divisor)
    m = len(dividendo) - n

    # 1) Normalización
    d = BASE // (divisor[-1] + 1)
    multiplicar_por_digito(dividendo, d)
    multiplicar_por_digito(divisor, d)
    # Asegurar espacio extra para el paso de resta
    if len(dividendo) < m + n + 1:
        dividendo += [0] * ((m + n + 1) - len(dividendo))

    # 2) Preparar lista del cociente
    cociente = [0] * (m + 1)

    # 3) Bucle principal: j = m … 0
    for j in range(m, -1, -1):
        # 3.1 Estimación inicial (dos dígitos más altos)
        tope = dividendo[j+n] * BASE + dividendo[j+n-1]
        coc_prueba = min(tope // divisor[-1], BASE - 1)

        # 3.2 Corrección como máximo dos veces
        if n > 1:
            resto_estim = tope % divisor[-1]
            while coc_prueba * divisor[-2] > resto_estim * BASE + dividendo[j+n-2]:
                coc_prueba -= 1
                resto_estim += divisor[-1]
                if resto_estim >= BASE:
                    break

        # 3.3 Restar el producto y, si hay borrow, corregir de nuevo
        if restar_producto(dividendo, j, divisor, coc_prueba):
            coc_prueba -= 1
            # Revertir un paso: sumar divisor desplazado
            acarreo = 0
            for k in range(n):
                suma = dividendo[j+k] + divisor[k] + acarreo
                dividendo[j+k] = suma % BASE
                acarreo = suma // BASE
            dividendo[j+n] += acarreo

        cociente[j] = coc_prueba

    # 4) Desnormalizar el resto
    resto_norm = dividendo[:n]
    acarreo = 0
    for i in range(n-1, -1, -1):
        cur = resto_norm[i] + acarreo * BASE
        resto_norm[i] = cur // d
        acarreo = cur % d

    # Quitar ceros altos
    while len(cociente) > 1 and cociente[-1] == 0:
        cociente.pop()
    while len(resto_norm) > 1 and resto_norm[-1] == 0:
        resto_norm.pop()

    return cociente, resto_norm

if __name__ == "__main__":
    A = cadena_a_digitos(input("Dividendo: "))
    Bv = cadena_a_digitos(input("Divisor:   "))
    Q, R = dividir_cociente_prueba(A, Bv)
    print("Cociente:", digitos_a_cadena(Q))
    print("Resto:   ", digitos_a_cadena(R))


Dividendo: 456
Divisor:   54
Cociente: 8
Resto:    24
